In [8]:
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime
from core.utils import Tibanna
from core import ff_utils

wf_md5 = "md5"
wf_fastqc = "fastqc-0-11-4-1/1"

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
run_md_qc = raw_input("Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)")

# status for completion
# there are two flavors of complete signals, before it was output_file_transfer_finished, not it is complete.
# old completed wf runs have former one.

ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

################
##ADD TO WORKFLOW
# wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
# run_hours = int((datetime.now()-wfr_time).total_seconds()/3600)
################

def get_files(exp_set_id):
    files = []
    exps = ff_utils.get_metadata(exp_set_id, connection=ff)['experiments_in_set']
    for an_exp in exps:
        exp_resp = ff_utils.get_metadata(an_exp, connection=ff)['files']
        files.extend(exp_resp)
    return files

def summarize_file(file_resp):
    md5 = False
    qc = False
    file_id = file_resp['accession']
    sequencer = file_resp.get('instrument')
    relations = file_resp.get('related_files')
    status = file_resp.get('status')
    workflows = file_resp.get('workflow_run_inputs')
    first_alias = file_resp.get('aliases',[None])[0]
    pair_no = file_resp.get('paired_end')
    # is md5 fine
    if file_resp.get('md5sum') and file_resp.get('content_md5sum'):
        md5 = True
    # is there a qc?
    if file_resp.get('quality_metric'):
        qc = True
    # Check workflows for qc fastqc workflow partA
    md5_status = 'did_not_run'
    fastqc_status = 'did_not_run'
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    if workflows:
        for a_wfr in workflows:
            wfr_resp = ff_utils.get_metadata(a_wfr['uuid'], connection=ff)
            wfr_name = wfr_resp['display_title']
            if wfr_name.startswith(wf_md5):
                md5_status = wfr_resp.get('run_status')     
            elif wfr_name.startswith(wf_fastqc):
                fastqc_status = wfr_resp.get('run_status')     
                
    # Check for md5 and fastqc, and if not complete, run or report it. 
    # if exclude miseq is on, do this only if sequencer is not miseq
  
    if not md5 or status == "uploading" or md5_status != 'complete':
        # if not, shall we run it?
        if run_md_qc in ['md5', 'all']:
            print 'md5 running for', file_resp['accession']
            code_md5= "invoke run_md5 " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(3)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_resp['accession']
    # check fastqc if md5 is fine
    else:
        if not qc or fastqc_status != 'complete':
            # if not, shall we run it?
            if run_md_qc in ['qc', 'all']:
                print 'fastqc running for', file_resp['accession']
                code_qc= "invoke run_fastqc " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
                print code_qc

                
                time.sleep(300)
                
                run(code_qc)
                print ''    
                time.sleep(300)
            # user does not want it to be run, so just report
            else:
                print 'fastqc run missing for', file_resp['accession'], fastqc_status
                print 
    return


all_sets = [
            'dcic:Selvaraj_gm12878_hic',
            'dekker-lab:ExperimentSet_U54_U54-ESC4DN-FA-DpnII-2017524'
            ]

#all_files = get_files(all_sets[1])


all_files = ff_utils.get_metadata('files-fastq', connection=ff)['@graph']
print len(all_files)


for a_file in all_files:  
    if a_file['status'] == 'deleted':
        #print "Deleted File", a_file
        continue
        
    file_resp = ff_utils.get_metadata(a_file['uuid'], connection=ff, frame='embedded')
    # check if file is in s3
    head_info = tibanna.s3.does_key_exist(file_resp['upload_key'], tibanna.s3.raw_file_bucket)
    if not head_info:
        print file_resp['accession'], "does not have a file in S3"
        continue
        
    file_info = summarize_file(file_resp)
    
 


Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)qc
2668
fastqc running for 4DNFI96XT4FW
invoke run_fastqc fourfront-webprod 4DNFI96XT4FW 7532829d-b761-42af-a112-18221a88651b


KeyboardInterrupt: 